<a href="https://colab.research.google.com/github/shiba01Git/Youtube_chatdat_Python/blob/main/twitter_DM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gspread oauth2client

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:

import time
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Google Sheets APIの設定
SCOPE = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
CREDS = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', SCOPE)
CLIENT = gspread.authorize(CREDS)

# スプレッドシートの読み込み
SPREADSHEET_ID = '1I1Myi3FmEO0J2BplvfRYKr-m1-yuFMmTBxeQ-zFZFfg'  # スプレッドシートID
SHEET_NAME = 'シート1'  # シート名
sheet = CLIENT.open_by_key(SPREADSHEET_ID).worksheet(SHEET_NAME)

# シートのデータを取得
data = sheet.get_all_records()
print(data)

# Seleniumの設定
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # ヘッドレスモード
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # Braveブラウザのパスを指定
    chrome_options.binary_location = "data/com.brave.browser/files/downloads"  # Braveのインストールパスを指

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

setup_driver()
print(driver)

# Twitterにログイン
def login_twitter(driver, username, password):
    driver.get("https://twitter.com/login")
    time.sleep(3)

    # ユーザー名入力
    username_field = driver.find_element(By.NAME, 'text')
    username_field.send_keys(username)
    driver.find_element(By.XPATH, "//span[text()='Next']").click()
    time.sleep(2)

    # パスワード入力
    password_field = driver.find_element(By.NAME, 'password')
    password_field.send_keys(password)
    driver.find_element(By.XPATH, "//span[text()='Log in']").click()
    time.sleep(3)

# DMを送信する
def send_dm(driver, recipient_username, message):
    driver.get(f"https://twitter.com/{recipient_username}")
    time.sleep(3)

    # メッセージボタンをクリック
    try:
        message_button = driver.find_element(By.XPATH, "//span[text()='Message']")
        message_button.click()
        time.sleep(3)

        # メッセージ送信
        dm_field = driver.find_element(By.XPATH, "//div[@role='textbox']")
        dm_field.send_keys(message)
        send_button = driver.find_element(By.XPATH, "//span[text()='Send']")
        send_button.click()
        time.sleep(2)
    except Exception as e:
        print(f"Error sending DM to {recipient_username}: {e}")

# メインの処理
def main():
    driver = setup_driver()

    for account in data:
        username = account['アカウントID']
        password = account['パスワード']
        recipient = account['送信先ユーザー名']
        message = account['送信メッセージ']

        try:
            # Twitterにログイン
            login_twitter(driver, username, password)

            # ランダムな間隔でDMを送信
            send_dm(driver, recipient, message)
            sleep_time = random.uniform(60, 120)  # 60〜120秒のランダムなスリープ
            print(f"Waiting for {sleep_time} seconds before next message.")
            time.sleep(sleep_time)

        except Exception as e:
            print(f"Error with account {username}: {e}")

    driver.quit()

#if __name__ == "__main__":
    main()

In [ ]:
import tweepy
import time

# Twitter APIの認証情報
BEARER_TOKEN = 'your_bearer_token_here'
API_KEY = 'your_api_key_here'
API_SECRET_KEY = 'your_api_secret_key_here'
ACCESS_TOKEN = 'your_access_token_here'
ACCESS_TOKEN_SECRET = 'your_access_token_secret_here'

# Twitter APIに接続
client = tweepy.Client(bearer_token=BEARER_TOKEN,
                       consumer_key=API_KEY,
                       consumer_secret=API_SECRET_KEY,
                       access_token=ACCESS_TOKEN,
                       access_token_secret=ACCESS_TOKEN_SECRET)

auth = tweepy.OAuth1UserHandler(API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

# 自動返信するメッセージ
REPLY_MESSAGE = "ご連絡ありがとうございます！こちらから改めてご案内します。"

# 受信済みDMのIDを保存して、同じメッセージには返信しないようにするためのリスト
replied_message_ids = set()

# DMを確認して返信する関数
def check_and_reply_dm():
    # DMのリストを取得
    dms = api.get_direct_messages(count=50)  # 最新50件のDMを取得

    for dm in dms:
        message_id = dm.id
        sender_id = dm.message_create['sender_id']
        message_text = dm.message_create['message_data']['text']

        # 自分が送ったメッセージには返信しない
        if sender_id == api.me().id:
            continue

        # 既に返信したメッセージはスキップ
        if message_id in replied_message_ids:
            continue

        # 自動返信
        try:
            api.send_direct_message(recipient_id=sender_id, text=REPLY_MESSAGE)
            print(f"Replied to DM from {sender_id}")

            # 返信済みのメッセージIDを保存
            replied_message_ids.add(message_id)
        except Exception as e:
            print(f"Error replying to DM: {e}")

# 定期的にDMをチェックして返信する
def main():
    while True:
        check_and_reply_dm()
        # 60秒待機して再度確認
        time.sleep(60)

if __name__ == "__main__":
    main()